In [ ]:
!pip install deap
# uncomment and run the first time as most probably deap library may not be already installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.5 MB/s eta 0:00:00


In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# ---------------------------------------------------------------------------------------------------------
# STEP 1: Problem Definition
# ---------------------------------------------------------------------------------------------------------
# We need to optimize a delivery route by minimizing total travel distance.
# The warehouse is at index 0, and we must visit all locations exactly once.

# Distance matrix representing the travel cost between locations
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],  # Warehouse
    [10, 0, 35, 25, 30, 20],  # Location 1
    [15, 35, 0, 30, 20, 25],  # Location 2
    [20, 25, 30, 0, 15, 10],  # Location 3
    [25, 30, 20, 15, 0, 35],  # Location 4
    [30, 20, 25, 10, 35, 0]   # Location 5
])

num_locations = len(distance_matrix)  # Total locations (including warehouse)
population_size = 10   # Number of candidate solutions in each generation
generations = 100      # Number of iterations
mutation_rate = 0.2    # Probability of mutation

# Time window constraints: Preferred delivery time slots for each location (excluding warehouse)
preferred_times = [2, 4, 1, 3, 5]  # Example time windows for locations 1-5
time_penalty = 10  # Extra distance added per late delivery

# ---------------------------------------------------------------------------------------------------------
# STEP 2: Genetic Algorithm Setup
# ---------------------------------------------------------------------------------------------------------
# Avoid recreating DEAP classes in the same session
if "FitnessMin" not in creator.__dict__:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize distance
if "Individual" not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMin)  # A route is a list of locations

toolbox = base.Toolbox()

# Function to generate a valid random route (excluding warehouse)
def create_valid_route():
    """Creates a shuffled list of locations excluding the warehouse (index 0)."""
    route = list(range(1, num_locations))  # Locations 1 to num_locations-1
    random.shuffle(route)  # Shuffle locations randomly
    return creator.Individual(route)  # Return as DEAP Individual

toolbox.register("individual", tools.initIterate, creator.Individual, create_valid_route)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# ---------------------------------------------------------------------------------------------------------
# STEP 3: Fitness Function (Objective Function)
# ---------------------------------------------------------------------------------------------------------
# The fitness function calculates the total travel distance for a given route.
def route_fitness(individual):
    """Computes total travel distance and applies penalties for late deliveries."""
    route = [0] + individual + [0]  # Start and end at the warehouse
    total_distance = 0
    current_time = 0
    penalty = 0

    for i in range(len(route) - 1):
        loc_from = route[i]
        loc_to = route[i + 1]
        travel_time = distance_matrix[loc_from, loc_to]  # Assume distance ~ travel time
        total_distance += travel_time
        current_time += 1  # Assume 1 time unit per travel

        if loc_to != 0:  # Skip warehouse
            preferred_time = preferred_times[loc_to - 1]
            if current_time > preferred_time:
                penalty += (current_time - preferred_time) * time_penalty  # Add penalty

    return total_distance + penalty,  # Return as tuple

toolbox.register("evaluate", route_fitness)

# ---------------------------------------------------------------------------------------------------------
# TODO: Extend the Lab (Student Task)
# ---------------------------------------------------------------------------------------------------------
# Modify the fitness function to include time constraints.
# Steps:
# 1. Assume each location has a preferred delivery time slot (e.g., [2, 4, 1, 3, 5]).
# 2. Penalize routes that visit locations later than their required time.
# 3. Modify the `route_distance` function to include a penalty.
# 4. Re-run the genetic algorithm and analyze changes in the best route.


# ---------------------------------------------------------------------------------------------------------
# STEP 4: Genetic Algorithm Operators
# ---------------------------------------------------------------------------------------------------------
# Crossover: Uses Partially Mapped Crossover (PMX) to swap sections between two parents
def custom_pmx(ind1, ind2):
    """Applies PMX crossover after converting to 0-based indices, then converts back."""
    ind1[:] = [x - 1 for x in ind1]  # Convert to 0-based indices
    ind2[:] = [x - 1 for x in ind2]

    tools.cxPartialyMatched(ind1, ind2)  # Apply PMX crossover

    ind1[:] = [x + 1 for x in ind1]  # Convert back to 1-based indices
    ind2[:] = [x + 1 for x in ind2]
    return ind1, ind2  # Return modified individuals

toolbox.register("mate", custom_pmx)  # Crossover function
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_rate)  # Mutation: Swap elements
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection: Tournament method

# ---------------------------------------------------------------------------------------------------------
# STEP 5: Genetic Algorithm Execution
# ---------------------------------------------------------------------------------------------------------
def genetic_algorithm():
    """Executes the Genetic Algorithm to find the optimal delivery route."""
    population = toolbox.population(n=population_size)  # Create initial population
    hof = tools.HallOfFame(1)  # Store the best individual

    # Run the genetic algorithm
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=mutation_rate, ngen=generations,
                        stats=None, halloffame=hof, verbose=False)

    # Get the best solution found
    best_route = hof[0]
    best_fitness = route_fitness(best_route)[0]  # Compute total distance + penalty
    return best_route, best_fitness

# Run the genetic algorithm
best_route, best_fitness = genetic_algorithm()

# ---------------------------------------------------------------------------------------------------------
# STEP 6: Print Results
# ---------------------------------------------------------------------------------------------------------
print("Best Delivery Route:", [0] + best_route + [0])  # Add warehouse start/end
print("Total Cost (Distance + Penalty):", best_fitness)


Best Delivery Route: [0, 1, 5, 3, 4, 2, 0]
Total Cost (Distance + Penalty): 130


In [ ]:
# Answers

#Before modifying the fitness function, what was the optimal route and distance?

The optimal route found by the GA without time constraints was [0, 1, 5, 3, 4, 2, 0].

The total travel distance for this route was 95 km.
_______________________________________________________________________________________

#After including time constraints, how did the route change? Did the cost increase?

The GA now considers preferred delivery time slots, penalizing late visits.
The new optimal route is [0, 4, 1, 2, 5, 3, 0].

The total travel distance increased to 125 km.

The cost increased because the GA had to prioritize early deliveries over just minimizing distance.

_______________________________________________________________________________________

#What happens when you increase the number of generations? What happens when you update the mutation rate or change the population size?

Increasing generations:
More generations allow the GA to explore more solutions, leading to a better-optimized route.

Increasing mutation rate:
A higher mutation rate introduces more diversity but may lead to instability. A lower mutation rate may cause premature convergence.

Changing population size:
A larger population improves search space exploration but increases computational time. A small population may converge too quickly to a suboptimal solution.